# Upload Payload Data and Trigger Evaluations Programatically

In [3]:
!pip install ibm_watson_openscale

In [4]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson_openscale import APIClient

In [5]:
service_credentials = {
                  "apikey": "YOUR_API_KEY",
                  "url": "https://api.aiopenscale.cloud.ibm.com"
               }

authenticator = IAMAuthenticator(
        apikey=service_credentials['apikey']
    )

# default value of service_url = "https://api.aiopenscale.cloud.ibm.com",
# service_instance_id retrieved automatically
client = APIClient(
        authenticator=authenticator
    )
client.version

'3.0.40'

In [6]:
DATAMART_ID = "bb92d5c7-da77-4244-b87f-285c120de28f"

In [7]:
SUBSCRIPTION_ID = "2041bac4-e42d-4c66-8386-ae4bb1d64a74"

## get the ids of monitor instances available for your subscription

In [8]:
monitor_instances_info  = client.monitor_instances.list(data_mart_id=DATAMART_ID, target_target_id=SUBSCRIPTION_ID)

In [23]:
monitor_instances_ids = []

for instance in monitor_instances_info .result.monitor_instances:
    try:
        print(instance.entity.schedule.status)
        print("Monitor :", instance.entity.monitor_definition_id)
        print("id :", instance.metadata.id)
        if(instance.entity.schedule.status == "enabled"):
            monitor_instances_ids.append(instance.metadata.id)
        print("-----------------------------------------------")
    except:
        pass

None
Monitor : drift_v2
id : 53e8bae8-5991-4499-8f39-a9b34acad87c
-----------------------------------------------
enabled
Monitor : model_health
id : 2d5ad376-cff4-462e-9db7-c7e9733a757e
-----------------------------------------------
enabled
Monitor : mrm
id : e10c8f83-81fb-4fc8-86e1-3a118625aadc
-----------------------------------------------
enabled
Monitor : generative_ai_quality
id : 13ff1aea-ecd0-493b-91a3-8bfdca728833
-----------------------------------------------


## Getting the ID of the payload dataset

In [10]:
data_sets = client.data_sets.list(target_target_id=SUBSCRIPTION_ID, target_target_type="subscription")

In [11]:
print(data_sets.result.data_sets[0].entity.type)

model_health


In [12]:
PAYLOAD_DATASET_ID = ""

for i in range(len(data_sets.result.data_sets)):
    print(data_sets.result.data_sets[i].entity.type)
    print(data_sets.result.data_sets[i].metadata.id)
    print("-----------------------------------------------")
    if(data_sets.result.data_sets[i].entity.type == "payload_logging"):
        PAYLOAD_DATASET_ID = data_sets.result.data_sets[i].metadata.id
    

model_health
217a47f0-1876-484f-89d6-d2a2482faa39
-----------------------------------------------
payload_logging
ee934aa5-6647-41bd-b888-a3b0bc72e216
-----------------------------------------------
feedback
b154cecf-0b98-43d5-bf3a-ee96f8b31a1c
-----------------------------------------------
gen_ai_quality_metrics
3da0e181-c1d9-4259-a568-c74155c7015f
-----------------------------------------------
payload_logging_error
684deed8-4146-4f55-ba8f-8b5289a3bad6
-----------------------------------------------
training
14dcdf87-a19c-49e5-832f-8918c51a832a
-----------------------------------------------
manual_labeling
6c6f4fad-c374-4ead-a56a-f7ce3b4de25f
-----------------------------------------------


## Getting the payload data

The below code gets the data from Cloud Object Storage and converts it to a dataframe as required by your payload data schema. 

In [13]:
import pandas as pd

In [14]:
# change this logic as per your data format and location
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='YOUR_API_KEY',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'custommetrics-donotdelete-pr-krgj8ymc0griyo'
object_key = 'summarisation.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_1 = pd.read_csv(body)
df_1.head(10)



,original_text,reference_output,generated_text
0,by using our services you are agreeing to thes...,by playing this game you agree to these terms....,by playing this game you agree to these terms....
1,if you want to use certain features of the ser...,you have to use google pokemon trainer club or...,you have to use google pokemon trainer club or...
2,during game play please be aware of your surro...,don t die or hurt others and if you do it s no...,don t die or hurt others and if you do it s no...
3,subject to your compliance with these terms ni...,don t copy modify resell distribute or reverse...,don t copy modify resell distribute or reverse...
4,for purposes of these terms a content means th...,we grant you full ownership of your user conte...,we grant you full ownership of your user conte...
5,the app permits account holders to capture and...,trading s gonna be a thing. don t try to bring...,trading s gonna be a thing. don t try to bring...
6,we may cancel suspend or terminate your accoun...,if you haven t played for a year you mess up o...,if you haven t played for a year you mess up o...
7,the app permits the purchase of virtual curren...,you can spend money on coins pokeballs. we ain...,you can spend money on coins pokeballs. we ain...
8,you agree that you are responsible for your ow...,don t be a jerk. don t hack or cheat. we don t...,don t be a jerk. don t hack or cheat. we don t...
9,we welcome feedback comments and suggestions f...,give us feedback and we can do whatever we wan...,give us feedback and we can do whatever we wan...


In [15]:
# change this logic as per your data format and location
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
request_body = {
    "request": {
      "fields": [
        "original_text",
      ],
      "values": []
    },
    "response": {
      "fields": [
        "original_text",
        "generated_text"
      ],
      "values": []
    }
}
  
for i in range(len(df_1)):
    request_body["request"]["values"].append([df_1.loc[i].original_text])
    request_body["response"]["values"].append([df_1.loc[i].original_text,df_1.loc[i].generated_text])
print(request_body)

{'request': {'fields': ['original_text'], 'values': [['by using our services you are agreeing to these terms our trainer guidelines and our privacy policy. if you are the parent or legal guardian of a child under the age of 13 the parent you are agreeing to these terms on behalf of yourself and your child ren who are authorized to use the services pursuant to these terms and in our privacy policy. if you don t agree to these terms our trainer guidelines and our privacy policy do not use the services.'], ['if you want to use certain features of the services you will have to create an account with us an account. you can create an account if you have a a pre existing google account b a pre existing pok\\u00e9mon trainer club ptc account c a pre existing account with facebook or d such other pre existing third party accounts as we may choose to support in the future notification of which will be provided by allowing selection of such pre existing accounts on the relevant account creation s

In [16]:
len(client.data_sets.get_list_of_records(data_set_id = PAYLOAD_DATASET_ID).result["records"])

20

In [17]:
## upload records to payload dataset

client.data_sets.store_records(data_set_id = PAYLOAD_DATASET_ID,  request_body= [PayloadRecord(request_body["request"], request_body["response"])], background_mode=False).result




 Waiting for end of storing records with request id: 209b2584-a068-43b8-8399-f94af5bf1f2f 




pending
active

---------------------------------------
 Successfully finished storing records 
---------------------------------------




In [18]:
len(client.data_sets.get_list_of_records(data_set_id = PAYLOAD_DATASET_ID).result["records"])

30

## Trigger evaluation for recently uploaded data

In [24]:
for monitor_instance_id in monitor_instances_ids:
    evaluation = client.monitor_instances.run(monitor_instance_id=monitor_instance_id, background_mode=False, triggered_by="user")
    print(evaluation.result)




 Waiting for end of monitoring run 9380cdaa-eff3-42f0-bb54-49bcf1731b82 




running
finished

---------------------------
 Successfully finished run 
---------------------------


{
  "metadata": {
    "id": "9380cdaa-eff3-42f0-bb54-49bcf1731b82",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/6e9ef5d9c71e47e78d4c75d7bdfdd804:bb92d5c7-da77-4244-b87f-285c120de28f:run:9380cdaa-eff3-42f0-bb54-49bcf1731b82",
    "url": "/v2/monitor_instances/2d5ad376-cff4-462e-9db7-c7e9733a757e/runs/9380cdaa-eff3-42f0-bb54-49bcf1731b82",
    "created_at": "2024-08-26T07:56:57.268000Z",
    "created_by": "IBMid-6660010MZ1"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "last_processed_record_scoring_timestamp": "2024-08-23T14:50:14.600000Z",
      "last_run_id": "3984250b-cde5-432d-8166-3cee165c9ae6",
      "last_run_measurement_id": null,
      "last_run_status": "FINISHED_WITHOUT_METRICS",
      "measurement_id": "4ca8250a-ac06-4e0e-9ce1-d57a21258fde",
      "run_me